In [2]:
!pip install pygame

In [5]:
import pygame
import random

# Inisialisasi Pygame
pygame.init()

# ========== Konstanta layar =============== 

# Mengatur ukuran layar.
# Membuat jendela game dengan lebar 800 dan tinggi 600 piksel.
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Jumping Game with Cliffs and Obstacles")

# Warna digunakan untuk objek-objek: langit, pemain, tebing, rintangan, dan layar kemenangan/kekalahan.
BLUE = (135, 206, 250)
WHITE = (255, 255, 255)
BROWN = (139, 69, 19)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# Kelas dasar Block
class Block:
    def __init__(self, x, y, width, height, color):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.color = color

    def draw(self, screen, camera_y):
        pygame.draw.rect(screen, self.color, (self.x, self.y - camera_y, self.width, self.height))

    def check_collision(self, other):
        return (
            self.x < other.x + other.width and
            self.x + self.width > other.x and
            self.y < other.y + other.height and
            self.y + self.height > other.y
        )

# Kelas Player (Mewarisi Block)
class Player(Block):
    def __init__(self, x, y):
        super().__init__(x, y, 50, 50, WHITE)
        self.vel_x = 5
        self.vel_y = 0
        self.gravity = 0.5
        self.is_jumping = False
        self.alive = True
        self.touched_first_cliff = False
    
    def move(self, keys, cliffs):
        if keys[pygame.K_LEFT]:
            self.x -= self.vel_x
        if keys[pygame.K_RIGHT]:
            self.x += self.vel_x
        if keys[pygame.K_SPACE] and not self.is_jumping:
            self.vel_y = -13
            self.is_jumping = True
        
        self.vel_y += self.gravity
        self.y += self.vel_y
        
        if self.y >= HEIGHT:
            self.alive = False  # Game Over jika jatuh ke bawah layar
        
        if self.x < 0:
            self.x = 0
        elif self.x > WIDTH - self.width:
            self.x = WIDTH - self.width
        
        # Cek tabrakan dengan tebing
        for cliff in cliffs[:]:  # Iterasi salinan daftar
            if (self.x + self.width > cliff.x and self.x < cliff.x + cliff.width and
                self.y + self.height > cliff.y and self.y + self.height - self.vel_y <= cliff.y):
                self.y = cliff.y - self.height
                self.is_jumping = False
                self.vel_y = 0
                if not self.touched_first_cliff and cliff != cliffs[0]:
                    self.touched_first_cliff = True
                    cliffs.pop(0)  # Hapus lantai dasar setelah pemain mencapai tebing pertama

# Kelas Tebing (Mewarisi Block)
class Cliff(Block):
    def __init__(self, x, y, width, height):
        super().__init__(x, y, width, height, BROWN)

# Kelas Rintangan (Mewarisi Block)
class Obstacle(Block):
    def __init__(self, x, y):
        super().__init__(x, y, 40, 40, RED)
        self.speed = 2
        self.direction = 1  # 1 ke kanan, -1 ke kiri

    def update(self, cliffs):
        # Hanya bergerak di tebing yang ada di bawahnya
        for cliff in cliffs:
            if self.y + self.height == cliff.y and self.x + self.width > cliff.x and self.x < cliff.x + cliff.width:
                self.x += self.speed * self.direction
                if self.x <= cliff.x or self.x + self.width >= cliff.x + cliff.width:
                    self.direction *= -1  # Balik arah saat sampai di ujung tebing
                break  # Pastikan hanya satu tebing yang dicek

# Inisialisasi pemain dan tebing
player = Player(WIDTH // 2, HEIGHT - 100)
cliffs = [Cliff(0, HEIGHT - 50, WIDTH, 50)]  # Lantai dasar
obstacles = []

# Tambahkan lebih banyak tebing secara acak
num_cliffs = 15
for i in range(1, num_cliffs + 1):
    x = random.randint(50, WIDTH - 200)
    y = HEIGHT - (i * 70)  # Setiap tebing naik 70 piksel ke atas
    width = random.randint(150, 250)
    height = 50
    new_cliff = Cliff(x, y, width, height)
    cliffs.append(new_cliff)

    # Tambahkan rintangan di beberapa tebing secara acak
    if random.random() < 0.4:  # 40% kemungkinan ada rintangan
        obstacle_x = x + random.randint(0, width - 40)
        obstacles.append(Obstacle(obstacle_x, y - 40))  # Di atas tebing

# Variabel kamera
camera_y = 0

# Tebing terakhir sebagai tujuan kemenangan
final_cliff = cliffs[-1]

# Loop utama
running = True
clock = pygame.time.Clock()
while running:
    clock.tick(30)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    keys = pygame.key.get_pressed()
    player.move(keys, cliffs)

    # Perbarui rintangan
    for obstacle in obstacles:
        obstacle.update(cliffs)

        # **Cek tabrakan antara pemain dan rintangan**
        if player.check_collision(obstacle):
            player.alive = False

    if not player.alive:
        screen.fill(RED)
        font = pygame.font.Font(None, 74)
        text = font.render("Game Over", True, WHITE)
        screen.blit(text, (WIDTH//2 - 100, HEIGHT//2 - 50))
        pygame.display.update()
        pygame.time.delay(2000)
        running = False
    elif player.y <= final_cliff.y:  # Pemain mencapai tebing terakhir (menang)
        screen.fill(GREEN)
        font = pygame.font.Font(None, 74)
        text = font.render("You Win!", True, WHITE)
        screen.blit(text, (WIDTH//2 - 100, HEIGHT//2 - 50))
        pygame.display.update()
        pygame.time.delay(2000)
        running = False
    else:
        # Update kamera agar mengikuti pemain ke atas
        if player.y < HEIGHT // 2:
            camera_y = player.y - HEIGHT // 2
        
        screen.fill(BLUE)
        for cliff in cliffs:
            cliff.draw(screen, camera_y)
        for obstacle in obstacles:
            obstacle.draw(screen, camera_y)
        player.draw(screen, camera_y)
    
    pygame.display.update()

# Hapus objek pemain sebelum keluar
del player
pygame.quit()


In [ ]:
# 7. Kelas Obstacle (Rintangan)

# class Obstacle(Block):
#     ...
# Rintangan merah yang bergerak bolak-balik di atas tebing.

# Logika update():

# Bergerak ke kanan/kiri di atas tebing.

# Saat mencapai ujung tebing, arah gerak dibalik (direction *= -1).

# 🔧 8. Inisialisasi Game

# player = Player(...)
# cliffs = [...]
# obstacles = []
# Pemain diletakkan di dekat bawah layar.

# Dibuat banyak tebing ke atas secara acak, tiap 70 piksel naik.

# Ada kemungkinan 40% setiap tebing akan punya rintangan.

# 🎥 9. Kamera

# camera_y = 0
# Kamera akan mengikuti pemain naik jika posisi Y pemain lebih tinggi dari tengah layar.

# 🔄 10. Loop Utama Game

# while running:
#     ...
# Input pemain diproses (kiri, kanan, lompat).

# Rintangan diperbarui dan dicek tabrakannya dengan pemain.

# Game Over jika:

# Pemain jatuh di bawah layar.

# Pemain menabrak rintangan.

# Menang jika mencapai tebing terakhir di atas.

# Kamera dan semua objek digambar ulang di layar sesuai posisi relatif terhadap kamera.

# 🧠 11. Mekanisme Penting
# Kamera:

# if player.y < HEIGHT // 2:
#     camera_y = player.y - HEIGHT // 2
# Kamera hanya naik jika pemain naik. Objek tetap di posisi aslinya, tapi digambar dengan offset -camera_y.

# Tabrakan Platform:

# self.y + self.height > cliff.y and self.y + self.height - self.vel_y <= cliff.y
# Pemain hanya mendarat jika datang dari atas, bukan dari samping/bawah.

# 🧹 12. Cleanup & Keluar

# pygame.quit()
# Setelah running = False, game ditutup dan memori dibersihkan.

# 🏁 13. Fitur-Fitur Menarik
# Fitur	Penjelasan
# Gravitasi	Pemain jatuh jika tidak di atas platform.
# Deteksi platform	Hanya mendarat jika pemain datang dari atas.
# Kamera mengikuti	Kamera mengikuti pemain ke atas.
# Rintangan cerdas	Bergerak otomatis dan membalik arah jika di ujung tebing.
# Lantai menghilang	Lantai dasar hilang setelah menyentuh tebing pertama.
# Kemenangan dan kalah	Tampilan visual berbeda saat menang/kalah.
# ✅ 14. Tips
# Demo dulu gamenya agar audience paham konteks.

# Jelaskan struktur kode berdasarkan fungsinya, bukan urutan penulisan.

# Gunakan analogi simpel: misalnya, “kamera seperti drone yang mengikuti pemain”.

# Tampilkan diagram seperti:

# Struktur objek: Player, Cliff, Obstacle.

# Alur game: Input → Gerakan → Cek tabrakan → Gambar ulang.
